In [2]:
import pandas as pd
import codecs
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter

In [22]:
newspapers = pd.read_csv("C:/Users/jw156/Ironhack/vaccine/vaccine_sentiment/english/covid_newspapers.csv", index_col=0)

In [23]:
newspapers

,title,href,date,new_title,id,corona_score,vaccine
1,guardian,https://www.theguardian.com/politics/2021/mar/...,2021-03-19,bori johnson receiv oxfordastrazeneca covid va...,politics,1.000000,NaN
3,guardian,https://www.theguardian.com/world/2021/feb/19/...,2021-02-19,sceptic oxford vaccin threaten europ s immunis...,world,1.000000,NaN
4,guardian,https://www.theguardian.com/world/2021/mar/18/...,2021-03-18,thursday brief EU s expert give oxford vaccin ...,world,1.000000,NaN
5,guardian,https://www.theguardian.com/world/2021/mar/16/...,2021-03-16,oxfordastrazeneca vaccin countri paus jab,world,1.000000,oxfordastrazeneca
6,guardian,https://www.theguardian.com/world/2021/mar/16/...,2021-03-16,chao germani itali suspens oxford vaccin,world,1.000000,NaN
...,...,...,...,...,...,...,...
32023,independent,https://www.independent.co.uk/news/education/e...,2021-03-25,not enough evid stop wear mask secondari schoo...,news,0.538462,NaN
32024,independent,https://www.independent.co.uk/news/world/ameri...,2021-03-25,brett favr reveal almost want kill amid painki...,news,0.555556,NaN
32025,independent,https://www.independent.co.uk/news/reports-say...,2021-03-25,report say cnn s chri cuomo got special covid ...,news,1.000000,NaN
32026,independent,https://www.independent.co.uk/news/uk/home-new...,2021-03-25,univers aberdeen becom one first UK institut r...,news,0.600000,NaN


In [24]:
newspapers.set_index("href", inplace=True)

In [25]:
newspapers = newspapers.reset_index(drop=True)

In [26]:
newspapers.title.drop_duplicates(inplace=True)

In [27]:
newspapers

,title,date,new_title,id,corona_score,vaccine
0,guardian,2021-03-19,bori johnson receiv oxfordastrazeneca covid va...,politics,1.000000,NaN
1,guardian,2021-02-19,sceptic oxford vaccin threaten europ s immunis...,world,1.000000,NaN
2,guardian,2021-03-18,thursday brief EU s expert give oxford vaccin ...,world,1.000000,NaN
3,guardian,2021-03-16,oxfordastrazeneca vaccin countri paus jab,world,1.000000,oxfordastrazeneca
4,guardian,2021-03-16,chao germani itali suspens oxford vaccin,world,1.000000,NaN
...,...,...,...,...,...,...
68403,independent,2021-03-25,not enough evid stop wear mask secondari schoo...,news,0.538462,NaN
68404,independent,2021-03-25,brett favr reveal almost want kill amid painki...,news,0.555556,NaN
68405,independent,2021-03-25,report say cnn s chri cuomo got special covid ...,news,1.000000,NaN
68406,independent,2021-03-25,univers aberdeen becom one first UK institut r...,news,0.600000,NaN


# POS Determination of words

In [28]:
newspapers = newspapers.reset_index(drop=True)

In [29]:
import nltk
from nltk.stem import WordNetLemmatizer 

In [30]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize


In [31]:
from nltk.corpus import wordnet as wn

def pos_headline(headline):
    new_headline = headline.split(' ')
    word_list = []
    for word in new_headline:
        if word == None or word.isalpha()==False:
            continue
        else:
            text = word_tokenize(word)
            postition_tag = nltk.pos_tag(text)
            word_list.append(postition_tag)
    return word_list

def is_proper_noun(tag):
    return tag in ['NNP', 'NNPS']

def is_noun(tag):
    return tag in ['NN', 'NNS']


def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']


def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']



def penn_to_wn(headlines):
    headline_list = pos_headline(headlines)
    tagged_list = []
    for tag in headline_list:
        tag = list(tag[0])
        if is_adjective(tag[1]):
            tag[1] = (wn.ADJ)
            tagged_list.append(tag)
        elif is_noun(tag[1]):
            tag[1] =(wn.NOUN)
            tagged_list.append(tag)
        elif is_adverb(tag[1]):
            tag[1] =(wn.ADV)
            tagged_list.append(tag)
        elif is_verb(tag[1]):
            tag[1] =(wn.VERB)
            tagged_list.append(tag)
        elif is_proper_noun(tag[1]):
            tag[1] =('NNP')
            tagged_list.append(tag)
    return tagged_list

def lemming_words(headline):
    if len(headline) == 0:
        return 0
    headline_list = penn_to_wn(headline)    
    new_string = []
    lemmatizer = WordNetLemmatizer()
    for word in headline_list:
        if word[1] == 'NNP':
            continue
        else:
            word[0] = (lemmatizer.lemmatize(word[0], word[1]))
            new_string.append(word)

    return new_string

In [32]:
x = newspapers.new_title[1]
lemming_words(x)


[['sceptic', 'a'],
 ['oxford', 'n'],
 ['vaccin', 'n'],
 ['threaten', 'n'],
 ['europ', 'n'],
 ['s', 'n'],
 ['immunis', 'n'],
 ['push', 'n']]

In [33]:
newspapers['pos_headline'] = newspapers['new_title'].apply(pos_headline)

In [34]:
newspapers['lem_pos_headline'] = newspapers['new_title'].apply(lambda x: lemming_words(x))

In [35]:
newspapers['pos_headline'][1]

[[('sceptic', 'JJ')],
 [('oxford', 'NN')],
 [('vaccin', 'NN')],
 [('threaten', 'NNS')],
 [('europ', 'NN')],
 [('s', 'NN')],
 [('immunis', 'NN')],
 [('push', 'NN')]]

# Word Sense Disambiguation

In [36]:
from nltk.wsd import lesk

x = 'The porn threatens to cause a lockdown'

print(lesk(x.split(), 'threatens', 'v'))


Synset('threaten.v.03')


In [37]:
def word_definitions(title, lemmed_title):
    synsets = []
    original_text = title
    lemmed_pos = lemmed_title
    for i in lemmed_pos:
        synsets.append(lesk(original_text.split(), i[0], i[1]))
    return synsets

In [39]:
# x = newspapers['title'][1]
# y = newspapers['lem_pos_headline'][1]
# import re
# from nltk.corpus import sentiwordnet as swn,SentiSynset

# j = word_definitions(x, y)
# k = str(j[0])
# print(k)
# x = (re.findall(r"'([^']*)'", k))
# print(x)
# f = swn.senti_synset(x[0])
# print(f)

In [40]:
newspapers['word_definitions'] = newspapers.apply(lambda x: word_definitions(x['title'], x['lem_pos_headline']), axis=1)


In [41]:
x = newspapers['word_definitions'][1]

In [42]:
for i in x:
    syn_word = (re.findall(r"'([^']*)'", str(i)))
    if len(syn_word) != 0:
        input_word = (syn_word[0])
        word_senti = (swn.senti_synset(input_word))
        pos = float(word_senti.pos_score()) 
        neg = float(word_senti.neg_score())
        overall_senti = pos + neg
        print(overall_senti)

0.0
0.0
0.0


## Save dataframe

In [43]:
# newspapers.to_csv('C:/Users/jw156/Ironhack/vaccine/newspaper_pos_score.csv')
# print("Sucessfully saved")

## Load dataframe

In [3]:
newspapers = pd.read_csv('C:/Users/jw156/Ironhack/vaccine/newspaper_pos_score.csv')
# newspapers.set_index(inplace=True)

In [4]:
newspapers

,hrefs,title,new_title,id,date,newspaper_name,vaccine,pos_headline,lem_pos_headline,word_definitions
0,https://www.theguardian.com/politics/2021/mar/...,Boris Johnson receives Oxford/AstraZeneca Covi...,bor johnson receiv oxfordastrazenec covid vaccin,politics,2021-03-19,guardian,NaN,"[[('Boris', 'NNP')], [('Johnson', 'NNP')], [('...","[['receives', 'n'], ['Covid', 'n'], ['vaccine'...","[None, None, Synset('vaccine.n.01')]"
1,https://www.theguardian.com/world/2021/feb/19/...,Scepticism over Oxford vaccine threatens Europ...,sceiv oxford vaccin threatens europ immun push,world,2021-02-19,guardian,NaN,"[[('Scepticism', 'NN')], [('over', 'IN')], [('...","[['Scepticism', 'n'], ['Oxford', 'n'], ['vacci...","[Synset('agnosticism.n.02'), Synset('oxford.n...."
2,https://www.theguardian.com/world/2021/mar/18/...,Thursday briefing: EU's experts to give Oxford...,thursday brief eu expert giv oxford vaccin ver...,world,2021-03-18,guardian,NaN,"[[('Thursday', 'NNP')], [('experts', 'NNS')], ...","[['expert', 'n'], ['give', 'v'], ['Oxford', 'n...","[Synset('expert.n.01'), Synset('give.v.18'), S..."
3,https://www.theguardian.com/world/2021/mar/16/...,Oxford/AstraZeneca vaccine: which countries ha...,oxfordastrazenec vaccin country paus jab,world,2021-03-16,guardian,NaN,"[[('which', 'WDT')], [('countries', 'NNS')], [...","[['country', 'n'], ['have', 'v'], ['pause', 'v...","[Synset('country.n.04'), Synset('take.v.35'), ..."
4,https://www.theguardian.com/world/2021/mar/16/...,Chaos in Germany and Italy after suspension of...,chao germany ita suspend oxford vaccin,world,2021-03-16,guardian,NaN,"[[('Chaos', 'NN')], [('in', 'IN')], [('Germany...","[['Chaos', 'n'], ['suspension', 'n'], ['Oxford...","[Synset('chaos.n.02'), Synset('suspension.n.05..."
...,...,...,...,...,...,...,...,...,...,...
62019,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[('Extreme', 'NN')], [('lockdown', 'NN')], [(...","[['Extreme', 'n'], ['lockdown', 'n'], ['law', ...","[Synset('extreme_point.n.01'), Synset('lockdow..."
62020,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[('Extreme', 'NN')], [('lockdown', 'NN')], [(...","[['Extreme', 'n'], ['lockdown', 'n'], ['law', ...","[Synset('extreme_point.n.01'), Synset('lockdow..."
62021,https://www.independent.co.uk/independentpremi...,The problem with Johnson’s vaccine passports f...,the problem johnson vaccin passport pub,business,2021-03-25,independent,NaN,"[[('The', 'DT')], [('problem', 'NN')], [('with...","[['problem', 'n'], ['vaccine', 'n'], ['passpor...","[Synset('problem.n.02'), Synset('vaccine.n.01'..."
62022,https://www.independent.co.uk/voices/editorial...,Reality is dawning on the EU: export bans are ...,real dawn eu export ban counterproduc cas covi...,uk,2021-03-25,independent,NaN,"[[('Reality', 'NN')], [('is', 'VBZ')], [('dawn...","[['Reality', 'n'], ['be', 'v'], ['dawn', 'v'],...","[Synset('reality.n.03'), Synset('embody.v.02')..."


# TF − IDF ∗ SentimentScore

In [5]:
test = pd.DataFrame(newspapers.iloc[0]).T

In [6]:
test

,hrefs,title,new_title,id,date,newspaper_name,vaccine,pos_headline,lem_pos_headline,word_definitions
0,https://www.theguardian.com/politics/2021/mar/...,Boris Johnson receives Oxford/AstraZeneca Covi...,bor johnson receiv oxfordastrazenec covid vaccin,politics,2021-03-19,guardian,NaN,"[[('Boris', 'NNP')], [('Johnson', 'NNP')], [('...","[['receives', 'n'], ['Covid', 'n'], ['vaccine'...","[None, None, Synset('vaccine.n.01')]"


In [7]:
N = len (newspapers)
processed_text = []

for i in newspapers.new_title[:N]:
    processed_text.append(i)

In [8]:
N

62024

In [9]:
# N = len (test)
# processed_title = []

# for i in test.new_title[:N]:
#     processed_text.append(i)

In [10]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    tokens = tokens.split(' ')
    for w in tokens:
        w = w.lower()
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [11]:
DF

{'bor': 3287,
 'johnson': 3531,
 'receiv': 221,
 'oxfordastrazenec': 47,
 'covid': 6023,
 'vaccin': 3927,
 'sceiv': 45,
 'oxford': 405,
 'threatens': 173,
 'europ': 740,
 'immun': 286,
 'push': 247,
 'thursday': 141,
 'brief': 958,
 'eu': 910,
 'expert': 884,
 'giv': 816,
 'verdict': 15,
 'country': 907,
 'paus': 66,
 'jab': 401,
 'chao': 284,
 'germany': 349,
 'ita': 451,
 'suspend': 212,
 'caut': 49,
 'sci': 950,
 'frant': 465,
 'press': 466,
 'shift': 70,
 'rollout': 284,
 'plan': 1685,
 'chang': 679,
 'follow': 350,
 'approv': 217,
 'belg': 75,
 'consid': 331,
 'uturn': 148,
 'over55s': 4,
 'slow': 210,
 'distribut': 48,
 'dos': 243,
 'cas': 2204,
 'ris': 1400,
 'debenham': 19,
 'shut': 319,
 'six': 598,
 'stor': 133,
 'includ': 127,
 'street': 336,
 'flag': 40,
 'spat': 23,
 'could': 2737,
 'lead': 1047,
 'diverg': 10,
 'strategies': 11,
 'what': 1108,
 'diff': 170,
 'mak': 988,
 'uk': 6472,
 'ful': 256,
 'reg': 295,
 'ms': 24,
 'redevelop': 5,
 'big': 360,
 'shop': 576,
 'mov': 4

In [12]:
total_vocab_size = len(DF)

In [13]:
total_vocab_size

15908

In [14]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

# Get Sentiment

In [106]:
from nltk.corpus import sentiwordnet as swn
import re

In [132]:
x = pd.DataFrame(newspapers.word_definitions[:1])
x

,word_definitions
0,"[None, None, Synset('vaccine.n.01')]"


In [135]:
def senti_score(definitions):
    from nltk.corpus import sentiwordnet as swn
    total = 0
    definitions = 
    for word in definitions:
        print(word)
#         if word != None:
#             x = (re.findall(r"'([^']*)'", word))
#             print(x)
#             f = float(swn.senti_synset(x[0]).pos_score()) + float(swn.senti_synset(x[0]).neg_score())
#             total += f
#     return total


In [136]:
x['senti_scores'] = x.word_definitions.apply(senti_score)

[
N
o
n
e
,
 
N
o
n
e
,
 
S
y
n
s
e
t
(
'
v
a
c
c
i
n
e
.
n
.
0
1
'
)
]


In [113]:
k = "Synset('vaccine.n.01')"
print(k)
x = (re.findall(r"'([^']*)'", k))
print(x)
f = swn.senti_synset(x[0])
print(f)

Synset('vaccine.n.01')
['vaccine.n.01']
<vaccine.n.01: PosScore=0.0 NegScore=0.0>


In [103]:
# senti_score("Synset('vaccine.n.01')")
print(swn.senti_synset('vaccine.n.01'))

<vaccine.n.01: PosScore=0.0 NegScore=0.0>


In [36]:
newspapers['sentiment'] = newspapers['word_definitions'].apply(senti_score)

ValueError: invalid literal for int() with base 10: "01')"

In [15]:
sentiment_dic={}

for i in index:
    sentiment_dic[df_senitment["word"][i].lower()]=(df_senitment["sentiment"][i])
    for word in df_senitment["word_forms"][i]:
        if len(word)>1:
            sentiment_dic[word.lower()]=(df_senitment["sentiment"][i])

3287

In [17]:
doc = 0
tf_idf = {}
for i in range(len(newspapers)):
    headline = processed_text[i]
    new_headline = headline.split(' ')
    counter = Counter(new_headline)
    words_count = len(new_headline)
    for token in np.unique(new_headline):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        tf_idf[doc, token] = tf*idf
    doc += 1

In [21]:
doc = 0
sentiment = {}
for i in range(N):
    headline = processed_text[i]
    new_headline = headline.split(' ')
    for token in np.unique(new_headline):
        try:
            sentiment_value=sentiment_dic[token]
            sentiment[doc, token] = sentiment_value
        except:
            sentiment[doc, token] = 0
    doc += 1

In [33]:
sentiment

{(0, 'bor'): 0,
 (0, 'covid'): 0,
 (0, 'johnson'): 0,
 (0, 'oxfordastrazenec'): 0,
 (0, 'receiv'): 0,
 (0, 'vaccin'): 0,
 (1, 'europ'): 0,
 (1, 'immun'): 0,
 (1, 'oxford'): 0,
 (1, 'push'): 0,
 (1, 'sceiv'): 0,
 (1, 'threatens'): 0,
 (1, 'vaccin'): 0,
 (2, 'brief'): 0,
 (2, 'eu'): 0,
 (2, 'expert'): 0,
 (2, 'giv'): 0,
 (2, 'oxford'): 0,
 (2, 'thursday'): 0,
 (2, 'vaccin'): 0,
 (2, 'verdict'): 0,
 (3, 'country'): 0,
 (3, 'jab'): 0,
 (3, 'oxfordastrazenec'): 0,
 (3, 'paus'): 0,
 (3, 'vaccin'): 0,
 (4, 'chao'): 0,
 (4, 'germany'): 0,
 (4, 'ita'): 0,
 (4, 'oxford'): 0,
 (4, 'suspend'): 0,
 (4, 'vaccin'): 0,
 (5, 'caut'): 0,
 (5, 'europ'): 0,
 (5, 'oxford'): 0,
 (5, 'sci'): 0,
 (5, 'vaccin'): 0,
 (6, 'covid'): 0,
 (6, 'frant'): 0,
 (6, 'germany'): 0,
 (6, 'oxford'): 0,
 (6, 'press'): 0,
 (6, 'shift'): 0,
 (6, 'vaccin'): 0,
 (7, 'approv'): 0,
 (7, 'chang'): 0,
 (7, 'follow'): 0,
 (7, 'oxfordastrazenec'): 0,
 (7, 'plan'): 0,
 (7, 'rollout'): 0,
 (7, 'vaccin'): 0,
 (8, 'belg'): 0,
 (8, 'consid

In [22]:
index_df=list(newspapers.index)

In [ ]:
newspapers

In [30]:
N = len (newspapers)

sum_sentiment={}

for index in index_df[:2]:
    headline = processed_text[index]
    new_headline = headline.split(' ')
#     print(new_headline)
    for token in np.unique(new_headline):
#         print(index)
#         print(token)
        if index in list(sum_sentiment.keys()):
            sentiment_value=float(sentiment[(index, token)])*float(tf_idf[(index, token)])
            print(sentiment_value)
            sum_sentiment[index] += sentiment_value
        else:
            sum_sentiment[index]=float(sentiment[(index, token)])*float(tf_idf[(index, token)])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [31]:
sum_sentiment

{0: 0.0, 1: 0.0}

In [25]:
newspapers["sentiment"]=newspapers.apply(lambda row: sum_sentiment[row.name],axis=1)

In [26]:
newspapers

,hrefs,title,new_title,id,date,newspaper_name,vaccine,pos_headline,lem_pos_headline,word_definitions,sentiment
0,https://www.theguardian.com/politics/2021/mar/...,Boris Johnson receives Oxford/AstraZeneca Covi...,bor johnson receiv oxfordastrazenec covid vaccin,politics,2021-03-19,guardian,NaN,"[[('Boris', 'NNP')], [('Johnson', 'NNP')], [('...","[['receives', 'n'], ['Covid', 'n'], ['vaccine'...","[None, None, Synset('vaccine.n.01')]",0.0
1,https://www.theguardian.com/world/2021/feb/19/...,Scepticism over Oxford vaccine threatens Europ...,sceiv oxford vaccin threatens europ immun push,world,2021-02-19,guardian,NaN,"[[('Scepticism', 'NN')], [('over', 'IN')], [('...","[['Scepticism', 'n'], ['Oxford', 'n'], ['vacci...","[Synset('agnosticism.n.02'), Synset('oxford.n....",0.0
2,https://www.theguardian.com/world/2021/mar/18/...,Thursday briefing: EU's experts to give Oxford...,thursday brief eu expert giv oxford vaccin ver...,world,2021-03-18,guardian,NaN,"[[('Thursday', 'NNP')], [('experts', 'NNS')], ...","[['expert', 'n'], ['give', 'v'], ['Oxford', 'n...","[Synset('expert.n.01'), Synset('give.v.18'), S...",0.0
3,https://www.theguardian.com/world/2021/mar/16/...,Oxford/AstraZeneca vaccine: which countries ha...,oxfordastrazenec vaccin country paus jab,world,2021-03-16,guardian,NaN,"[[('which', 'WDT')], [('countries', 'NNS')], [...","[['country', 'n'], ['have', 'v'], ['pause', 'v...","[Synset('country.n.04'), Synset('take.v.35'), ...",0.0
4,https://www.theguardian.com/world/2021/mar/16/...,Chaos in Germany and Italy after suspension of...,chao germany ita suspend oxford vaccin,world,2021-03-16,guardian,NaN,"[[('Chaos', 'NN')], [('in', 'IN')], [('Germany...","[['Chaos', 'n'], ['suspension', 'n'], ['Oxford...","[Synset('chaos.n.02'), Synset('suspension.n.05...",0.0
...,...,...,...,...,...,...,...,...,...,...,...
62019,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[('Extreme', 'NN')], [('lockdown', 'NN')], [(...","[['Extreme', 'n'], ['lockdown', 'n'], ['law', ...","[Synset('extreme_point.n.01'), Synset('lockdow...",0.0
62020,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[('Extreme', 'NN')], [('lockdown', 'NN')], [(...","[['Extreme', 'n'], ['lockdown', 'n'], ['law', ...","[Synset('extreme_point.n.01'), Synset('lockdow...",0.0
62021,https://www.independent.co.uk/independentpremi...,The problem with Johnson’s vaccine passports f...,the problem johnson vaccin passport pub,business,2021-03-25,independent,NaN,"[[('The', 'DT')], [('problem', 'NN')], [('with...","[['problem', 'n'], ['vaccine', 'n'], ['passpor...","[Synset('problem.n.02'), Synset('vaccine.n.01'...",0.0
62022,https://www.independent.co.uk/voices/editorial...,Reality is dawning on the EU: export bans are ...,real dawn eu export ban counterproduc cas covi...,uk,2021-03-25,independent,NaN,"[[('Reality', 'NN')], [('is', 'VBZ')], [('dawn...","[['Reality', 'n'], ['be', 'v'], ['dawn', 'v'],...","[Synset('reality.n.03'), Synset('embody.v.02')...",0.0
